<li><span><a href="#For category prediction" data-toc-modified-id="For category prediction-2"><span class="toc-item-num"></span>1. GRU (loss type: TOP1)</a></span>

<li><span><a href="#ItemKNN and SKNN"
data-toc-modified-id="ItemKNN and SKNN-1.1"><span class="toc-item-num">2.&nbsp;&nbsp;</span>Change loss type (BPR loss)</a></span></li>
<li><span><a href="#ItemKNN and SKNN"
data-toc-modified-id="ItemKNN and SKNN-1.1"><span class="toc-item-num">3.&nbsp;&nbsp;</span>Change loss type (Cross-Entropy loss)</a></span></li>

</ul></div>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
path="drive/My\ Drive/1001_project_Total"

In [3]:
!python $path/Data/preprocess.py --Path $path/Data/day_one_article_region_device.csv --Feature True --Try True

Full train set
	Events: 187978
	Sessions: 66718
	Items: 385
Test set
	Events: 35574
	Sessions: 11746
	Items: 267
Train set
	Events: 140275
	Sessions: 50721
	Items: 357
Validation set
	Events: 18508
	Sessions: 6735
	Items: 237


# GRU (loss type: TOP1)

In [0]:
lr=[0.01,0.02,0.03]
hidden_size=[500,750,1000]

In [0]:
import sys
import warnings
warnings.simplefilter('ignore')
sys.path.append('drive/My Drive/pyGRU4REC_copy')
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from data import SessionDataset
from model import GRU4REC

In [0]:
#-----------data
train = 'day_one_train_full.txt'
test = 'day_one_test_full.txt'

train_dataset = SessionDataset(train)
test_dataset = SessionDataset(test, itemmap=train_dataset.itemmap)
#----------parameters
input_size = len(train_dataset.items)
num_layers =1
output_size = input_size
batch_size = 50
optimizer_type = 'Adagrad'
weight_decay = 0
momentum = 0
eps = 1e-6
n_epochs = 5
use_cuda = True
torch.manual_seed(7)

def run(hidd_size,l,loss_type):
  for hidden_size in hidd_size:
    for lr in l:
      model = GRU4REC(input_size, hidden_size, output_size,
                      num_layers=num_layers,
                      batch_size=batch_size,
                      optimizer_type=optimizer_type,
                      lr=lr,
                      weight_decay=weight_decay,
                      momentum=momentum,
                      eps=eps,
                      loss_type=loss_type,
                      use_cuda=use_cuda)
      model_name = 'GRU4REC'
      print("hidden size is {0}, lr is {1}".format(hidden_size,lr))
      model.train(train_dataset, n_epochs=n_epochs, model_name=model_name, save=True)
      model.test(test_dataset)
      print("\n")

In [9]:
print("Loss type is TOP1: \n")
loss_type = 'TOP1'
run(hidden_size,lr,loss_type)

Loss type is TOP1: 

hidden size is 500, lr is 0.01
Training GRU4REC...
epoch: 1/loss:0.920/recall:0.690/mrr:0.296/time:0.131
epoch: 1/loss:0.903/ recall:0.708/ mrr:0.304/ time:0.128
The end of training~


Test result: loss:1.044/ recall:0.651/ mrr:0.237/ time:0.013


hidden size is 500, lr is 0.02
Training GRU4REC...
epoch: 1/loss:0.912/recall:0.687/mrr:0.289/time:0.128
The end of training~


Test result: loss:1.042/ recall:0.660/ mrr:0.239/ time:0.013


hidden size is 500, lr is 0.03
Training GRU4REC...
epoch: 1/loss:0.910/recall:0.677/mrr:0.283/time:0.127
The end of training~


Test result: loss:1.040/ recall:0.658/ mrr:0.231/ time:0.013


hidden size is 750, lr is 0.01
Training GRU4REC...
epoch: 1/loss:0.917/recall:0.696/mrr:0.296/time:0.128
epoch: 1/loss:0.901/ recall:0.705/ mrr:0.302/ time:0.128
The end of training~


Test result: loss:1.046/ recall:0.666/ mrr:0.234/ time:0.013


hidden size is 750, lr is 0.02
Training GRU4REC...
epoch: 1/loss:0.911/recall:0.685/mrr:0.288/time:0.

## 2. Change loss type(BPR Loss)

In [10]:
loss_type = 'BPR'
run(hidden_size,lr,loss_type)

hidden size is 500, lr is 0.01
Training GRU4REC...
epoch: 1/loss:0.472/recall:0.655/mrr:0.276/time:0.119
epoch: 1/loss:0.418/ recall:0.707/ mrr:0.296/ time:0.119
epoch: 2/loss:0.407/ recall:0.708/ mrr:0.297/ time:0.120
epoch: 3/loss:0.401/ recall:0.712/ mrr:0.299/ time:0.119
epoch: 4/loss:0.397/ recall:0.715/ mrr:0.301/ time:0.119


Test result: loss:0.688/ recall:0.647/ mrr:0.225/ time:0.012


hidden size is 500, lr is 0.02
Training GRU4REC...
epoch: 1/loss:0.443/recall:0.669/mrr:0.276/time:0.117
epoch: 1/loss:0.401/ recall:0.711/ mrr:0.297/ time:0.120
epoch: 2/loss:0.392/ recall:0.717/ mrr:0.300/ time:0.121
epoch: 3/loss:0.387/ recall:0.722/ mrr:0.300/ time:0.123
epoch: 4/loss:0.384/ recall:0.726/ mrr:0.301/ time:0.121


Test result: loss:0.671/ recall:0.669/ mrr:0.231/ time:0.013


hidden size is 500, lr is 0.03
Training GRU4REC...
epoch: 1/loss:0.433/recall:0.671/mrr:0.274/time:0.125
epoch: 1/loss:0.396/ recall:0.712/ mrr:0.295/ time:0.119
epoch: 2/loss:0.387/ recall:0.721/ mrr:0.2

## 3. Change loss type(Cross-Entropy loss)

In [11]:
loss_type = 'CrossEntropy'
run(hidden_size,lr,loss_type)

hidden size is 500, lr is 0.01
Training GRU4REC...
epoch: 1/loss:3.388/recall:0.612/mrr:0.268/time:0.114
epoch: 1/loss:3.237/ recall:0.675/ mrr:0.297/ time:0.115
epoch: 2/loss:3.204/ recall:0.680/ mrr:0.300/ time:0.116
epoch: 3/loss:3.186/ recall:0.682/ mrr:0.300/ time:0.115
epoch: 4/loss:3.174/ recall:0.683/ mrr:0.301/ time:0.114


Test result: loss:3.881/ recall:0.621/ mrr:0.224/ time:0.012


hidden size is 500, lr is 0.02
Training GRU4REC...
epoch: 1/loss:3.298/recall:0.635/mrr:0.276/time:0.115
epoch: 1/loss:3.182/ recall:0.684/ mrr:0.297/ time:0.116
epoch: 2/loss:3.158/ recall:0.687/ mrr:0.298/ time:0.121
epoch: 3/loss:3.143/ recall:0.690/ mrr:0.297/ time:0.116
epoch: 4/loss:3.133/ recall:0.692/ mrr:0.297/ time:0.114


Test result: loss:3.852/ recall:0.648/ mrr:0.227/ time:0.013


hidden size is 500, lr is 0.03
Training GRU4REC...
epoch: 1/loss:3.275/recall:0.642/mrr:0.274/time:0.118
epoch: 1/loss:3.167/ recall:0.686/ mrr:0.294/ time:0.116
epoch: 2/loss:3.143/ recall:0.691/ mrr:0.2